# HW 5：詞頻矩陣 - 管中閔近期臉書貼文

## 背景

台大校長遴選爭議延燒近三個月，準校長管中閔只能偶爾在臉書上發文略表無奈。為了一探管爺這幾個月來的心理狀態，我從管爺的臉書上複製 2018年1月15日 至 2018年3月28日 的所有貼文內容，除了文字貼文外，也將圖片及分享連結內的文字內容一併複製下來存進 txt檔中，經文字處理後計算 TF-IDF 做近一步的分析。

## 小結

1. TF-IDF：15篇貼文中，僅有8篇有高權重 TF-IDF 的字詞 (> 0.5)，推測其他貼文內容相關性高，因此無法區分出重要的字詞。
2. Cosine Similarity：以管中閔於3月22日發的臉書聲明為基準，計算各個貼文相對此篇的餘弦相似性，其中餘閒相似度最高的為政大法學院副教授廖元豪於風傳媒上的評論〈廖元豪觀點：別讓台大校長遴選案歹戲拖棚〉。
3. Visualization：比較上述兩篇文章的文字雲，從管中閔臉書聲明的「遲遲」、「懈怠」、「准否」等詞，可以看出管爺心急如焚，也正正呼應了廖元豪一文中一再呼籲「教育部」別再「拖」了的訴求。

--------------------------------------------------------------------------------------------------------------------------------
## Load Library

In [1]:
import pandas as pd
import jieba
jieba.load_userdict("CMKuan_dict.txt")

import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jestw\AppData\Local\Temp\jieba.cache
Loading model cost 1.585 seconds.
Prefix dict has been built succesfully.


## Import Data

In [2]:
file = open('./files/tfidf_test.txt', encoding="utf8")

## TD-IDF

### Step 1: Create Corpus

In [3]:
corpus = []
for line in file:
    corpus.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))

In [4]:
corpus

["\ufeff 與 大春 結交 逾   45   載 ， 首得 大春 如此 賜書 ， 驚喜 萬分 。 信中言 最近 之 事 ， 雖僅 數語 ， 但 已嚴 於 斧 鉞 ； 更 感謝 老友 信中 最 後 一句 的 鼓勵 之意 。 期待 風波 平靜 後 ， 能 與 大春 和 其他 諸友 再度 飲酒 唱和 。 ' \n",
 "一 、 大學 校長 遴選 ， 依大學法 為 校長 遴選委員會 職權 。 二 、 今年 一月 五日 臺灣大學 校長 遴選委員會 依法 選任 本人 管中閔 為 校長 當選人 ， 並報 請 教育部 聘任 ， 迄今已 逾 兩個 半月 。 三 、 政府 各部 門與政 媒勢力 不 斷 製 造 莫須有 的 不實 指控 ， 教育部 再 配合 推諉 懈怠 ， 遲遲 不 對 該 聘任 案 作出 准否 之 核示 ， 已使 臺灣大學 校務 擘劃 、 決策 與 年度 財務 規劃 產生 困難 。 四 、 特此 聲明 ， 敬請 教育部 於 三月底 前 、 對 臺 大校 長 遴選委員會 報請 聘任 本人 為 臺灣大學 校長 一事 ， 應為 准否 之 函示 ' \n",
 "想 擺脫 這種 攻擊 ， 簡直 沒有 辦法 。 深感 羞辱 之下 ， 我 準備 要 「 答 復 」 那 一大堆 無稽 的 指控 和 那些 下流 惡毒 的 造謠 。 可是 我始終 沒 完成 這個 工作 。 隔天 早上 ， 又 有 一個 報紙 刊出 一個 新 的 恐怖事件 ， 再度 惡 意中 傷 ， 嚴厲 控訴 我 燒毀 了 一個 瘋人院 ， 院裡 的 病人 全 被 燒死 了 ， 只 因 它 妨礙 了 我 的 住宅 的 視野 。 這使 我 陷落 恐慌 之中 。 然後又來 個 控訴 ， 說 我 曾 為 奪取 叔父 的 財產 、 把 他 毒死 了並 提出 緊急 要求 ， 要開 挖 墳 墓驗屍 。 這簡直 把 我 嚇到 快 到 發瘋 。 這 一切 還不夠 呢 ， 又給 我 按 上 一個 新 的 罪名 ， 說 我 在 棄嬰 收養 所 當所長 的 時候 ， 曾經 雇用 了 一些 掉 光牙齒 、 老邁 無能 的 親戚 擔任 烹飪 工作 。 我 開始 動搖 了 ， 一一 動搖 了 。 最 後 ， 黨派 相爭 的 仇恨 加到 我 身上 的 無恥 迫害 ， 終於 很 自然 發展 到 了 高潮 ： 九個 剛學 走路 的 小孩子 ， 包

### Step 2: Build TF-IDF Matrix

In [5]:
# stop words for tfidf
stopwords = ['我', '對','與',' ','的','在','是','有','了','說','她','也','就','不','和','臺','長','為','去','之','後','於','但','人','一個','他','來','最','都','而','到']
puncs = "！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟'〰〾〿–—‘’‛“”„‟…‧﹏.。"

In [6]:
# tfidf
vectorizer = TfidfVectorizer(min_df=1, stop_words = stopwords) # punctuation is ignored by default with TfidfVectorizer
tfidf = vectorizer.fit_transform(corpus)

In [7]:
print("tfidf.shape: ", tfidf.shape)
tfidf

tfidf.shape:  (15, 1096)


<15x1096 sparse matrix of type '<class 'numpy.float64'>'
	with 1238 stored elements in Compressed Sparse Row format>

### Step 3: Score Each Document

高權重的 TF-IDF 取決於特定文件內的高詞頻，及該詞語跨文件的低文件出現率 (high term frequency in the given document and a low document frequency of the term in the whole collection of documents)。透過計算 TF-IDF 並篩選出數值大於 0.5 者，我們可以得知每個貼文中的重要字詞。15篇貼文中，僅有8篇有高權重 TF-IDF 的字詞 (> 0.5)，推測其他貼文內容相關性高，因此無法區分出重要的字詞。

In [8]:
words = vectorizer.get_feature_names()

for i in range(len(corpus)):
    print('----Document {0}----'.format(i))
    for j in range(len(words)):
        if tfidf[i,j] > 0.5:
              print(words[j], tfidf[i,j], i ,j)

----Document 0----
大春 0.522462788918 0 332
----Document 1----
----Document 2----
----Document 3----
----Document 4----
教育部 0.547097303505 4 536
----Document 5----
----Document 6----
不要 0.550924717454 6 54
----Document 7----
對花 0.516122602195 7 381
最誠摯 0.516122602195 7 562
蓮獻上 0.516122602195 7 882
----Document 8----
----Document 9----
價值 0.509784776077 9 148
是不是 0.509784776077 9 554
----Document 10----
莫須有 1.0 10 875
----Document 11----
----Document 12----
am 0.707106781187 12 4
fine 0.707106781187 12 5
----Document 13----
照片 0.62735102629 13 692
----Document 14----


### Step 4: Compute Cosine Similarity

以管中閔於3月22日發的臉書聲明 (Document 1) 為基準，以 `cosine_similarity` 計算各個貼文相對此篇的餘弦相似性。其中，餘閒相似度最高的為 Document 4，此篇為政大法學院副教授廖元豪於風傳媒上的評論 [廖元豪觀點：別讓台大校長遴選案歹戲拖棚](http://www.storm.mg/article/406022)。以下分別輸出兩篇文章以供檢視及比較。

In [9]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = tfidf_vectorizer.fit_transform(corpus)  #finds the tfidf score with normalization
print ("cosine scores\n ", cosine_similarity(tfidf_matrix_train[1], tfidf_matrix_train))

cosine scores
  [[ 0.          1.          0.01061691  0.          0.30245055  0.          0.
   0.          0.          0.02639221  0.09110556  0.04386519  0.          0.
   0.0425827 ]]


In [10]:
Document_1 = corpus[1]
Document_1 = Document_1.replace(" ","")
print("管中閔臉書聲明：\n\n", Document_1)

管中閔臉書聲明：

 一、大學校長遴選，依大學法為校長遴選委員會職權。二、今年一月五日臺灣大學校長遴選委員會依法選任本人管中閔為校長當選人，並報請教育部聘任，迄今已逾兩個半月。三、政府各部門與政媒勢力不斷製造莫須有的不實指控，教育部再配合推諉懈怠，遲遲不對該聘任案作出准否之核示，已使臺灣大學校務擘劃、決策與年度財務規劃產生困難。四、特此聲明，敬請教育部於三月底前、對臺大校長遴選委員會報請聘任本人為臺灣大學校長一事，應為准否之函示'



In [11]:
Document_4 = corpus[4]
Document_4 = Document_4.replace(" ","")
print("廖元豪觀點：\n\n", Document_4)

廖元豪觀點：

 各大學已經開學超過一週，號稱台灣第一的國立臺灣大學，卻至今沒有校長，而且會繼續拖多久都沒人知道。對此歹戲拖棚，教育部恐怕要負最大的責任。為什麼？因為台大早已依據大學法相關程序，遴選出管中閔教授擔任校長並送請教育部遴聘。但教育部既未遴聘，卻也不敢拒絕遴聘。只會不斷地發文要求台大澄清澄清再澄清，要求人家補充說明許多根本無須補充與說明的事項。公文來來去去，但就是看不到教育部自己的意見—到底遴選程序是否合法？管中閔先生到底合格不合格？的確，教育部在大學校長遴選的過程中，並不是橡皮圖章。大學法第九條規定，雖然校長是由遴選委員會遴選，但仍要報請教育部來聘任。教育部雖然不能干涉遴選委員會的實質判斷，但仍有「合法性監督」的權限。也就是說，如果教育部確實發現台大校長的遴選過程，有違法之處，當然可以具體敘明理由，拒絕聘任。一旦拒絕聘任，而台大就必須補正或重啟遴選程序。問題是，教育部到底認為這次的程序，合法或違法呢？20180221-台大自主行動聯盟上午舉辦「台大人不服從向小英學姊拜年」活動，最後前往教育部，由部長潘文忠表達抗議。圖為參與的民眾隔著教育部的欄杆，將訴求折成紙飛機射進教育部。（蘇仲泓攝）台大校長遴選產生已經滿兩個月，但教育部始終始終拖著不予核備。圖為台大自主行動聯盟舉辦「台大人不服從向小英學姊拜年」活動，圖為參與的民眾隔著教育部的欄杆，將訴求折成紙飛機射進教育部。（蘇仲泓攝）問題就是，教育部沒說話，卻連下了七道金牌，反覆要求遴選委員會與台大澄清「獨董爭議」與「學術倫理」的事項。然後在2月又召開學術審議委員會，要求台大校方說明。前幾天甚至還冒出第八道「金牌密令」，跳過公文程序，用電子郵件向台大人事室調閱管中閔的兼職資料。公文往返，來來去去就拖了一個多月。然而，這些爭議需要靠公文旅行來釐清嗎？管中閔先生擔任台哥大獨董，以及所謂學術倫理的爭議，在「事實」方面早就清清楚楚。有什麼需要進一步靠公文來調查的？這兩件爭議是否構成程序上的瑕疵，而它們有無影響遴選結果的合法性，都是「法律判斷」的問題，而沒有什麼需要調查的事實。也就是說，針對這些「事實」，教育部根本就可以直接認定合法或違法，然後負起責任—認定合法，就立即聘任；認為已經到了違法之地步，就拒絕核定，不予聘任。以此事件的政治爭議性來看，不管教育部怎樣認定，都一定會有後續的政治與法律衝突。尤其是拒絕核定，不予聘任，還

## Visualization

除了直接檢視兩篇文章外，我們也可以透過繪製文字雲，將兩篇文章的內容作視覺化的呈現及比較。

### Step 1: Create a Word Dictionary

+ 管中閔臉書聲明

In [12]:
seg_list = jieba.cut(Document_1, cut_all=False)  # 精确模式
seg_list = list(seg_list)
wordDict_1 = {}
for w in seg_list:
    if (w not in stopwords) and (w not in puncs):
        if w in wordDict_1:
            wordDict_1[w] = wordDict_1[w] + 1
        else:
            wordDict_1[w] = 1
print(wordDict_1)

{'一': 1, '大學': 1, '校長': 5, '遴選': 1, '依大學法': 1, '遴選委員會': 3, '職權': 1, '二': 1, '今年': 1, '一月': 1, '五日': 1, '臺灣大學': 3, '依法': 1, '選任': 1, '本人': 2, '管中閔': 1, '當選人': 1, '並報': 1, '請': 1, '教育部': 3, '聘任': 3, '迄今已': 1, '逾': 1, '兩個': 1, '半月': 1, '三': 1, '政府': 1, '各部': 1, '門與政': 1, '媒勢力': 1, '斷': 1, '製': 1, '造': 1, '莫須有': 1, '不實': 1, '指控': 1, '再': 1, '配合': 1, '推諉': 1, '懈怠': 1, '遲遲': 1, '該': 1, '案': 1, '作出': 1, '准否': 2, '核示': 1, '已使': 1, '校務': 1, '擘劃': 1, '決策': 1, '年度': 1, '財務': 1, '規劃': 1, '產生': 1, '困難': 1, '四': 1, '特此': 1, '聲明': 1, '敬請': 1, '三月底': 1, '前': 1, '大校': 1, '報請': 1, '一事': 1, '應為': 1, '函示': 1, '\n': 1}


In [13]:
result_1 = pd.DataFrame({'key':[x for x in wordDict_1.keys()],'value':[x for x in wordDict_1.values()]})
result_1 = result_1.sort_values(by='value',ascending=False)
result_1 = result_1.reset_index(drop=True)
result_1.head(10)

,key,value
0,校長,5
1,聘任,3
2,教育部,3
3,遴選委員會,3
4,臺灣大學,3
5,准否,2
6,本人,2
7,校務,1
8,財務,1
9,年度,1


In [14]:
result_1.to_csv("Doc1_wordcloud.csv", encoding='utf-8')

+ 廖元豪觀點

In [15]:
seg_list = jieba.cut(Document_4, cut_all=False)  # 精确模式
seg_list = list(seg_list)
wordDict_4 = {}
for w in seg_list:
    if (w not in stopwords) and (w not in puncs):
        if w in wordDict_4:
            wordDict_4[w] = wordDict_4[w] + 1
        else:
            wordDict_4[w] = 1
print(wordDict_4)

{'各大學': 1, '已經': 2, '開學': 1, '超過': 1, '一週': 1, '號稱': 1, '台灣': 1, '第一': 1, '國立': 1, '臺灣大學': 2, '卻': 3, '至今': 1, '沒有': 1, '校長': 6, '而且': 2, '會繼續': 1, '拖': 4, '多久': 1, '沒人': 1, '知道': 1, '對此': 1, '歹戲': 1, '拖棚': 1, '教育部': 25, '恐怕': 1, '要負': 1, '最大': 1, '責任': 2, '什麼': 3, '因為': 1, '台大': 16, '早已': 1, '依據': 1, '大學': 4, '法相': 1, '關': 1, '程序': 6, '遴選出': 1, '管中閔': 4, '教授': 1, '擔': 1, '任校': 1, '並送': 1, '請': 3, '遴聘': 3, '既': 1, '未': 1, '不敢': 2, '拒絕': 5, '只會': 1, '不斷': 1, '地': 1, '發文': 1, '要求': 4, '澄清': 4, '再': 1, '人家': 1, '補充': 2, '明許': 1, '多': 1, '根本': 3, '無須': 1, '說明': 2, '事項': 3, '公文': 4, '來來': 2, '就是': 4, '看不到': 1, '自己': 2, '意見': 1, '到底': 3, '遴選': 8, '是否': 2, '合法': 4, '先生': 2, '合格': 2, '的確': 1, '過程': 2, '中': 1, '並': 1, '不是': 4, '橡皮': 1, '圖章': 1, '大學法': 1, '第九': 1, '條規定': 1, '雖然': 2, '由': 2, '遴選委員會': 4, '仍': 2, '要': 2, '報': 1, '聘任': 6, '不能': 1, '干涉': 2, '實質': 1, '判斷': 2, '合法性': 2, '監督': 1, '權限': 1, '如果': 1, '確實': 1, '發現': 1, '台': 2, '大校': 2, '違法': 4, '之處': 1, '當然': 1, '可以': 4, '具體': 1, '敘明': 1, '

In [25]:
result_4 = pd.DataFrame({'key':[x for x in wordDict_4.keys()],'value':[x for x in wordDict_4.values()]})
result_4 = result_4.sort_values(by='value',ascending=False)
result_4 = result_4.reset_index(drop=True)
result_4.head(20)

,key,value
0,教育部,25
1,台大,16
2,遴選,8
3,認定,8
4,程序,6
5,聘任,6
6,爭議,6
7,校長,6
8,拒絕,5
9,就是,4


In [26]:
result_4.to_csv("Doc4_wordcloud.csv", encoding='utf-8')

＊ 因為 Window 10 始終無法成功執行 wordcloud package，只好將詞頻存成 csv檔丟進 Tableau 繪製文字雲。

### Step 2: Create Word Cloud

比較兩篇文章的文字雲，描述事實的用詞如「校長」、「教育部」、「遴選」、「台大」等出現頻率最高，原因可想而知。比較有趣的是，從管中閔臉書聲明的「遲遲」、「懈怠」、「准否」等詞，可以看出管爺心急如焚，也正正呼應了廖元豪一文中一再呼籲「教育部」別再「拖」了的訴求。

In [1]:
%%HTML
<script type='text/javascript' src='https://us-west-2b.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1000px; height: 827px;'><object class='tableauViz' width='1000' height='827' style='display:none;'><param name='host_url' value='https%3A%2F%2Fus-west-2b.online.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='&#47;t&#47;jessiecreates' /><param name='name' value='CMKuanWordcloud_2&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /><param name='filter' value='iframeSizedToWindow=true' /></object></div>